In [ ]:
# # 코랩 javascript 멈춤 방지 코드
# var startClickConnect = function startClickConnect(){
#     var clickConnect = function clickConnect(){
#         console.log("Connnect Clicked - Start");
#         document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
#         console.log("Connnect Clicked - End");
#     };

#     var intervalId = setInterval(clickConnect, 60000*30);

#     var stopClickConnectHandler = function stopClickConnect() {
#         console.log("Connnect Clicked Stopped - Start");
#         clearInterval(intervalId);
#         console.log("Connnect Clicked Stopped - End");
#     };

#     return stopClickConnectHandler;
# };

# var stopClickConnect = startClickConnect();

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
import random
import torch

from pykospacing import Spacing
import re

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# KF-DeBERTa 모델
model_name = 'kakaobank/kf-deberta-base'
model = AutoModel.from_pretrained(model_name)
Classification_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 2진분류
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
data_df = pd.read_excel('./data/New_samsung_news.xlsx')

In [5]:
data_df.loc[data_df['Outcome'] == '악재', 'labels'] = 0
data_df.loc[data_df['Outcome'] == '호재', 'labels'] = 1
data_df['labels'] = data_df['labels'].astype(int)

In [6]:
# 띄어쓰기, 대소문자 보정 함수
spacing = Spacing()
def preprocessing(text):
    text = spacing(text)
    text = text.lower()  # 소문자 변경
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 기본 불용어 불러오기
korean_stopwords_path = "data/stopwords-ko.txt"
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

# # 불용어 처리 함수
# def remove_stopwords(text, stopwords):
#     words = text.split()
#     filtered_text = []
#     for word in words:
#         if text not in stopwords:
#             filtered_text.append(text)
#     return ' '.join(filtered_text)

# 불용어 처리 함수 수정
def remove_stopwords(text, stopwords):
    words = text.split()  # 문장을 단어로 분리
    filtered_text = [word for word in words if word not in stopwords]  # 단어가 불용어 목록에 없는 경우만 추가
    return ' '.join(filtered_text)  # 필터링된 단어들 다시 조합

In [ ]:
import nlpaug.augmenter.word as naw
import nltk
nltk.download('punkt')

# 문장을 단위로 분리하고, 각 문장에 단어 순서 변경 적용
def augment_paragraph(paragraph):
    # 문단을 문장으로 분리
    sentences = nltk.sent_tokenize(paragraph)
    aug = naw.RandomWordAug(action="swap", aug_p=0.3)  # aug_p로 변경할 비율 설정 (30%)

    augmented_sentences = []
    
    for sentence in sentences:
        # 마침표로 끝나는지 확인하고, 있으면 따로 저장
        has_period = sentence.endswith('.')
        
        # 단어 순서 변경 적용
        augmented_sentence = aug.augment(sentence)[0]
        
        # 중복 마침표 제거 및 마침표 유지
        augmented_sentence = augmented_sentence.rstrip('.')
        if has_period:
            augmented_sentence += '.'
        
        augmented_sentences.append(augmented_sentence)

    # 문단으로 다시 조합
    return ' '.join(augmented_sentences)

In [ ]:
# 증강 수행
for i in (range(len(data_df))):
    feature_text = data_df.loc[i, 'summary_content']  # 해당 행의 텍스트 가져오기
    augmented_text = augment_paragraph(feature_text)
    
    # 해당 행의 'processed' 컬럼에 저장
    data_df.loc[i, 'change_text'] = augmented_text

In [ ]:
data_df.to_excel('./data/data_samsung_change.xlsx')

# 저장된 엑셀파일 수기로 수정

In [ ]:
# 문장 전처리 및 불용어 처리
for i in tqdm(range(len(data_df))):
    feature_text = data_df.loc[i, 'summary_content']
    processed_text = preprocessing(feature_text)
    cleaned_text = remove_stopwords(processed_text, stopwords)
    data_df.loc[i, 'processed'] = cleaned_text

In [ ]:
data_df.to_excel('./data/data_samsung_processed.xlsx')
# data_df = pd.read_excel('./data/data_samsung_processed.xlsx')

In [24]:
# KF_DeBERT모델 텍스트 토큰화
encodings = tokenizer(data_df['processed'].tolist(), truncation=True, padding=True)  # truncation=최대길이를 초과하는 것에 대한 처리
data_df['encoding'] = encodings

# 데이터셋 준비
data_df['input_ids'] = encodings['input_ids']  # input_ids=문장을 토크나이저가 일정 단위로 쪼개서 vocab에 들어있는 숫자로 치환한 것
data_df['attention_mask'] = encodings['attention_mask']  # attention_mask=attention대상인지 아닌지를 나타냄. 대상이면1, 아니면0, 패딩된건 0으로 학습대상x

In [ ]:
data_df.to_excel('./data/data_apple_encoding.xlsx')
# data_df = pd.read_excel('./data/data_apple_encoding.xlsx')

# # 문자열을 리스트로 변환
# data_df['input_ids'] = data_df['input_ids'].str.strip('[]').str.split(', ').apply(lambda x: list(map(int, x)))
# data_df['attention_mask'] = data_df['attention_mask'].str.strip('[]').str.split(', ').apply(lambda x: list(map(int, x)))

In [ ]:
# 데이터 비중 조정
total_sample = len(data_df)
negative = data_df[data_df['labels']==0]  # 부정(악재) 클래스 5,849
positive = data_df[data_df['labels']==1]  # 긍정(호재) 클래스 10,672

# 데이터 비율 설정
total_rate = 1
negative_rate = 1.60
positive_rate = 0.80
negative_sample_size = round(len(negative) * total_rate * negative_rate)
positive_sample_size = round(len(positive) * total_rate * positive_rate)
print(negative_sample_size)
print(positive_sample_size)

In [ ]:
# 데이터 선택
negative_data = random.sample(list(negative.index), negative_sample_size-len(negative))
add_negative = list(pd.concat([negative, negative.loc[negative_data]]).index)
# negative_data = list(negative.index)
positive_data = list(positive.index)

# 최종 데이터 인덱스
sample_index = positive_data + add_negative
# sample_index = positive_data + negative_data
random.shuffle(sample_index)

# 데이터프레임 생성
sample_df = data_df.loc[sample_index]

# train_test_split 데이터set
train_columns = sample_df[['summary_content', 'input_ids', 'attention_mask']]
test_colums = sample_df[['labels']]

X_train, X_test, y_train, y_test = train_test_split(train_columns, test_colums, test_size=0.2)

train_dataset = Dataset.from_pandas(X_train.join(y_train))
test_dataset = Dataset.from_pandas(X_test.join(y_test))

In [28]:
# 하이퍼파라미터 그리드 정의
param_grid = {
    'learning_rate': [5e-5],  # 2e-5, 3e-5, 5e-5
    'per_device_train_batch_size': [8],
    'num_train_epochs': [3]
}

# # 하이퍼파라미터 옵션 설정
# param_grid = {
#     'learning_rate': [3e-5, 2e-5, 5e-5],  # 학습률
#     'per_device_train_batch_size': [4, 8, 16, 32],  # 베치크기
#     'num_train_epochs': [2, 4, 6, 8]  # 에포크 수
# }

# Best Parameters: {'learning_rate': 3e-05, 'batch_size': 16, 'num_train_epochs': 8}
# Best Accuracy: 0.55

In [ ]:
# 수치 초기화
best_accuracy = 0
best_F1 = 0
best_params = None
results = []

for learning_rate in tqdm(param_grid['learning_rate']):
    for batch_size in param_grid['per_device_train_batch_size']:
        for epochs in param_grid['num_train_epochs']:
            try:
                if batch_size > len(train_dataset):
                    print(f"Skipping batch size {batch_size} as it is larger than the dataset size {len(train_dataset)}")
                    continue

                # TrainingArguments 설정
                training_args = TrainingArguments(
                    output_dir="./results",  # 학습된 모델과 결과를 저장할 경로 설정
                    run_name="my_experiment",  # 고유한 run_name 설정  // 로그인 피하기위해
                    eval_strategy="epoch",  # 각 에포크마다 평가 수행
                    learning_rate=learning_rate,  # 학습률 설정
                    per_device_train_batch_size=batch_size,  # 학습 배치 크기 설정
                    per_device_eval_batch_size=batch_size,  # 평가 배치 크기 설정
                    num_train_epochs=epochs,  # 현재 학습 에포크 수 설정
                    weight_decay=0.01,  # 가중치 감쇠 설정
                    logging_dir='./logs',  # 로그 저장 경로 설정
                    logging_steps=10,  # 로그를 기록할 단계 수 설정
                )

                # Trainer 생성
                trainer = Trainer(
                    # model=Classification_model,
                    model=Classification_model.to(device),  # 모델을 GPU로 이동
                    args=training_args,
                    train_dataset=train_dataset,  # 훈련 데이터셋
                    eval_dataset=test_dataset,  # 평가 데이터셋
                )

                # 모델 학습
                trainer.train()

                # 훈련 데이터에서 평가
                train_results = trainer.predict(train_dataset)
                train_preds = np.argmax(train_results.predictions, axis=1)
                train_labels = train_results.label_ids
                train_accuracy = accuracy_score(train_labels, train_preds)
                train_precision = precision_score(train_labels, train_preds, average='binary')
                train_recall = recall_score(train_labels, train_preds, average='binary')
                train_f1 = f1_score(train_labels, train_preds, average='binary')
                train_loss = train_results.metrics['eval_loss'] if 'eval_loss' in train_results.metrics else None

                # 테스트 데이터에서 평가
                eval_results = trainer.evaluate()
                predictions = trainer.predict(test_dataset)
                preds = np.argmax(predictions.predictions, axis=1)
                labels = predictions.label_ids
                eval_accuracy = accuracy_score(labels, preds)
                eval_precision = precision_score(labels, preds, average='binary')
                eval_recall = recall_score(labels, preds, average='binary')
                eval_f1 = f1_score(labels, preds, average='binary')
                eval_loss = eval_results['eval_loss'] if 'eval_loss' in eval_results else None
                tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
                eval_specificity = tn / (tn + fp)

                # 성능 메트릭 계산
                accuracy = accuracy_score(labels, preds)
                precision = precision_score(labels, preds, average='binary')
                recall = recall_score(labels, preds, average='binary')
                f1 = f1_score(labels, preds, average='binary')
                tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
                specificity = tn / (tn + fp)

                # 결과 저장
                results.append({
                    'learning_rate': learning_rate,
                    'batch_size': batch_size,
                    'num_train_epochs': epochs,
                    'train_accuracy': train_accuracy,
                    'train_precision': train_precision,
                    'train_recall': train_recall,
                    'train_f1': train_f1,
                    'train_loss': train_loss,
                    'eval_accuracy': eval_accuracy,
                    'eval_precision': eval_precision,
                    'eval_recall': eval_recall,
                    'eval_f1': eval_f1,
                    'eval_loss': eval_loss,
                    'eval_specificity': eval_specificity
                })

                # 최고 성능 모델 기록
                if eval_accuracy > best_accuracy:
                    best_accuracy = eval_accuracy
                    best_F1 = eval_f1
                    best_params = {
                        'learning_rate': learning_rate,
                        'batch_size': batch_size,
                        'num_train_epochs': epochs
                    }

            except Exception as e:
                print(f"Error with parameters: learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
                print(f"Exception: {e}")
                continue


# 최적의 하이퍼파라미터 조합 출력
print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)
print("Best F1:", best_F1)

In [ ]:
def overfit_or_underfit(results):
    for result in results:
        train_accuracy = result['train_accuracy']
        eval_accuracy = result['eval_accuracy']

        # 과적합: 훈련 정확도와 평가 정확도의 차이가 크면 과적합
        if train_accuracy - eval_accuracy > 0.1:
            status = "과적합"

        # 과소적합: 훈련 및 평가 정확도가 모두 낮으면 과소적합
        elif train_accuracy < 0.7 and eval_accuracy < 0.7:
            status = "과소적합"

        # 정상: 성능 차이가 적당하고 평가 정확도가 높으면 정상 학습
        else:
            status = "정상"

        # 결과 출력
        print(f"Learning Rate: {result['learning_rate']}, Batch Size: {result['batch_size']}, Epochs: {result['num_train_epochs']}")
        print(f"Train Accuracy: {train_accuracy:.4f}, Eval Accuracy: {eval_accuracy:.4f}")
        print(f"Status: {status}\n")


# 함수 실행
overfit_or_underfit(results)

In [ ]:
conf_matrix = confusion_matrix(labels, preds)

# 혼동행렬 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# 모든 결과 출력
for result in results:
    print(result)

In [ ]:
from sklearn.metrics import classification_report

# # 최종 모델 평가
final_eval_results = trainer.evaluate()

# classification_report를 계산하고 딕셔너리로 변환
report_dict = classification_report(labels, preds, output_dict=True)

# 딕셔너리를 pandas 데이터프레임으로 변환
report_df = pd.DataFrame(report_dict).transpose()

# 데이터프레임을 주피터 노트북에서 보기 좋게 출력
styled_report = report_df.style.format("{:.2f}").background_gradient(cmap='Blues')

# 최종 평가 결과와 스타일링된 classification_report 출력
print(f"Final Accuracy: {accuracy}")
styled_report

In [ ]:
# # 수치 초기화
# best_accuracy = 0
# best_params = None
# results = []

# for learning_rate in tqdm(param_grid['learning_rate']):
#     for batch_size in param_grid['per_device_train_batch_size']:
#         for epochs in param_grid['num_train_epochs']:
#             try:
#                 if batch_size > len(train_dataset):
#                     print(f"Skipping batch size {batch_size} as it is larger than the dataset size {len(train_dataset)}")
#                     continue

#                 # TrainingArguments 설정
#                 training_args = TrainingArguments(
#                     output_dir="./results",  # 학습된 모델과 결과를 저장할 경로 설정
#                     evaluation_strategy="epoch",  # 각 에포크마다 평가 수행
#                     learning_rate=learning_rate,  # 학습률 설정
#                     per_device_train_batch_size=batch_size,  # 학습 배치 크기 설정
#                     per_device_eval_batch_size=batch_size,  # 평가 배치 크기 설정
#                     num_train_epochs=epochs,  # 현재 학습 에포크 수 설정
#                     weight_decay=0.01,  # 가중치 감쇠 설정
#                     logging_dir='./logs',  # 로그 저장 경로 설정
#                     logging_steps=10,  # 로그를 기록할 단계 수 설정
#                 )

#                 # Trainer 생성
#                 trainer = Trainer(
#                     model=Classification_model,  # 훈련모델 설정
#                     args=training_args,
#                     train_dataset=train_dataset,  # 훈련 데이터셋
#                     eval_dataset=test_dataset,  # 평가 데이터셋
#                 )

#                 # 모델 학습
#                 trainer.train()

#                 # 모델 평가
#                 eval_results = trainer.evaluate()

#                 # 예측 수행
#                 predictions = trainer.predict(test_dataset)
#                 preds = np.argmax(predictions.predictions, axis=1)  # 모델이 예측한 클래스의 인덱스를 포함하는 배열
#                 labels = predictions.label_ids  # 실제 정답 레이블의 인덱스를 포함하는 배열

#                 # 평가지표
#                 accuracy = accuracy_score(labels, preds)
#                 precision = precision_score(labels, preds, average='binary')
#                 recall = recall_score(labels, preds, average='binary')
#                 f1 = f1_score(labels, preds, average='binary')
#                 tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
#                 specificity = tn / (tn + fp)

#                 results.append({
#                     'learning_rate': learning_rate,
#                     'batch_size': batch_size,
#                     'num_train_epochs': epochs,
#                     'accuracy': accuracy,
#                     'precision': precision,
#                     'recall': recall,
#                     'specificity': specificity,
#                     'f1_score': f1
#                 })

#                 # 정확도를 기준으로 최고모델 저장
#                 if accuracy > best_accuracy:
#                     best_accuracy = accuracy
#                     best_params = {
#                         'learning_rate': learning_rate,
#                         'batch_size': batch_size,
#                         'num_train_epochs': epochs
#                     }
#                     best_confusion_matrix = confusion_matrix(labels, preds)

#             except Exception as e:
#                 print(f"Error with parameters: learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
#                 print(f"Exception: {e}")
#                 continue  # 에러 발생 시 다음 파라미터 조합으로 넘어감

# # 최적 하이퍼파라미터 조합과 정확도 출력
# print("Best Parameters:", best_params)
# print("Best Accuracy:", best_accuracy)

In [ ]:
# # 혼동행렬 시각화
# plt.figure(figsize=(8, 6))
# sns.heatmap(best_confusion_matrix, annot=True, fmt='d', cmap='Blues')
# plt.title('Confusion Matrix')
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.show()

# # 모든 결과 출력
# for result in results:
#     print(result)

In [ ]:
# # 최적 하이퍼파라미터로 다시 학습
# best_training_args = TrainingArguments(
#     output_dir='./best_model',
#     learning_rate=best_params['learning_rate'],
#     per_device_train_batch_size=best_params['per_device_train_batch_size'],
#     num_train_epochs=best_params['num_train_epochs'],
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
# )

# best_trainer = Trainer(
#     model=model,
#     args=best_training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset
# )

# best_trainer.train()
# best_trainer.save_model('./best_model')

In [ ]:
# 모델 저장
model.save_pretrained("./kf-deberta-finetuned")
tokenizer.save_pretrained("./kf-deberta-finetuned")